In [11]:
import json
from os import path
import os
import sys
import glob
sys.path.append("/home/shtoshni/Research/long-doc-coref/src")

from transformers import BertTokenizer
from inference.inference import Inference

In [12]:
input_dir = "/home/shtoshni/Research/events/proc_data/kbp_2015/cleaned"
input_files = glob.glob(path.join(input_dir, "*.jsonlines"))
input_files = [filename for filename in input_files if 'train.512' in filename]
print(input_files)

output_dir = "/home/shtoshni/Research/events/proc_data/kbp_2015/entity_chains"
if not path.exists(output_dir):
    os.makedirs(output_dir)
model_loc = "/home/shtoshni/Research/long-doc-coref/models/umem_ontonotes/model.pth"

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

['/home/shtoshni/Research/events/proc_data/kbp_2015/cleaned/train.512.jsonlines']


In [13]:
model = Inference(model_loc, device='cpu')
# model = Inference(model_loc)

In [14]:
# model.perform_coreference("Kanye West was released from prison. He was caught with drugs.")

In [15]:
for input_file in input_files:
    output_file = path.join(output_dir, path.basename(input_file))
    
    with open(input_file) as input_f, open(output_file, "w") as output_f:
        for line in input_f:
            instance = json.loads(line.strip())
            doc = ""
            doc_len = 0
            word_idx_to_subtoken_maps = {}
            word_counter = 0
            for sentence in instance["tokenized_sentences"]:
                doc += " ".join([word for word, _ in sentence]) + " "
                doc_len += len(sentence)
                for _, subtoken_span in sentence:
                    word_idx_to_subtoken_maps[word_counter] = subtoken_span
                    word_counter += 1
                
            doc = doc.strip()
            output_dict = model.perform_coreference(doc)
            
            all_tokens = []
            cur_subtoken_list = []
            cur_idx = 0
            
            all_subtokens = []
            for sentence in output_dict["tokenized_doc"]["sentences"]:
                all_subtokens.extend(sentence)
                
            for idx, subtoken_idx in enumerate(output_dict["tokenized_doc"]["subtoken_map"]):
                if subtoken_idx == cur_idx:
                    if all_subtokens[idx] not in ['[SEP]', '[CLS]']:
                        cur_subtoken_list.append(all_subtokens[idx])
                else:
                    all_tokens.append(bert_tokenizer.convert_tokens_to_string(cur_subtoken_list))
                    if all_subtokens[idx] not in ['[SEP]', '[CLS]']:
                        cur_subtoken_list = [all_subtokens[idx]]
                    else:
                        cur_subtoken_list = []
                    cur_idx = subtoken_idx
            
            
            if cur_subtoken_list != []:
                all_tokens.append(bert_tokenizer.convert_tokens_to_string(cur_subtoken_list))
                         
            assert (doc_len == len(all_tokens))
            clusters = [cluster for cluster in output_dict["clusters"] if len(cluster) > 1]
            entity_clusters = []
            for cluster in clusters:
                entity_clusters.append([
                    (word_idx_to_subtoken_maps[start_idx][0], word_idx_to_subtoken_maps[end_idx][1], entity_str) 
                    for (start_idx, end_idx), entity_str in cluster])
                
                
                    
#             print(entity_clusters)
            
            instance["entity_clusters"] = entity_clusters
            output_f.write(json.dumps(instance) + "\n")
#             break
            
        